In [1]:
### CLTV Prediction wiith BG-NBD and Gamma-Gamma ###

In [2]:
### WORK ISSUES ###

#The brand wants to create roadmap for sales and marketing. 
#The company's need to make a for medium-long term plan existing customers to the company 
#in the future so that they can estimating the potential value they will provide required. 

In [3]:
#The dataset includes brand's last purchases from OmniChannel (both online and offline shoppers) 
#in 2020 - 2021.
#It consists of information obtained from the past shopping behavior of customers who

In [4]:
### QUEST 1 ###
### Preparing the data ###

In [10]:
pip install lifetimes

Note: you may need to restart the kernel to use updated packages.


In [11]:
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions
from sklearn.preprocessing import MinMaxScaler

In [7]:
#Step 1
#Read dataset
df = pd.read_csv(r"C:\Users\PC\Desktop\Miuul_VBO\FLO_CLTV_Tahmini-221020-121219\FLO_CLTV_Tahmini\flo_data_20k.csv")
df2 = df.copy

In [ ]:
# Step 2
# Define the outlier_thresholds and replace_with_thresholds functions needed to suppress outliers.
#Note: When calculating cltv, frequency values must be integers. 
#Therefore, round the lower and upper limits with round()


In [16]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = round(low_limit,0)
    dataframe.loc[(dataframe[variable] > up_limit), variable] = round(up_limit,0)

In [17]:
# Step 3 
#"order_num_total_ever_online", "order_num_total_ever_offline", "customer_value_total_ever_offline",
# "customer_value_total_ever_online" If the  variables have outliers, suppress them.

In [18]:
columns = ["order_num_total_ever_online", "order_num_total_ever_offline", "customer_value_total_ever_offline","customer_value_total_ever_online"]
for col in columns:
    replace_with_thresholds(df, col)

In [19]:
#Step 4
#Omnichannel means that customers shop from both online and offline platforms.
#Total for each customer create new variables for the number of purchases and spending.

In [20]:
df["order_num_total"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"] 
df["customer_value_total"] = df["customer_value_total_ever_online"] + df["customer_value_total_ever_offline"]

In [21]:
#Step 4 
#Examine variable types. Change types of variables which showing the date

In [34]:
date_columns = df.columns[df.columns.str.contains("date")]
df[date_columns] = df[date_columns].apply(pd.to_datetime)

In [23]:
### QUEST 2 ###

#Creating CLTV Data Structure

In [24]:
#Step 1
#Take 2 days after the date of the last purchase in the data set as the analysis date.

In [25]:
df.columns

Index(['master_id', 'order_channel', 'last_order_channel', 'first_order_date',
       'last_order_date', 'last_order_date_online', 'last_order_date_offline',
       'order_num_total_ever_online', 'order_num_total_ever_offline',
       'customer_value_total_ever_offline', 'customer_value_total_ever_online',
       'interested_in_categories_12', 'order_num_total',
       'customer_value_total'],
      dtype='object')

In [26]:
df["last_order_date"].max()

'2021-05-30'

In [27]:
analysis_date = dt.datetime(2021, 6, 2)

In [28]:
#Step 2 
#Create a new cltv dataframe with customer_id, recency_cltv_weekly, T_weekly, frequency
#and monetary_cltv_avg values.
#Monetary value will be expressed as average value per purchase, recency and tenure values 
#will be expressed in weekly terms.

In [36]:
cltv_df = pd.DataFrame()
cltv_df["customer_id"] = df["master_id"]
cltv_df["recency_cltv_weekly"] = ((df["last_order_date"] - df["first_order_date"]).astype("timedelta64[D]")) / 7
cltv_df["T_weekly"] = ((analysis_date - df["first_order_date"]).astype('timedelta64[D]'))/7
cltv_df["frequency"] = df["order_num_total"]
cltv_df["monetary_cltv_avg"] = df["customer_value_total"] / df["order_num_total"]

cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.000000,30.714286,5.0,187.874000
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.857143,225.000000,21.0,95.883333
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.285714,79.000000,5.0,117.064000
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.571429,21.000000,2.0,60.985000
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.142857,95.571429,2.0,104.990000


In [37]:
### QUEST 3 ###
# Establishment of BG/NBD, Gamma-Gamma Models and Calculation of CLTV

In [40]:
# Step 1 
# Fit the BG/NBD Model

#Estimate expected purchases from customers in 3 months and cltv as exp_sales_3_month 
#and add it to the dataframe.
 
#Estimate expected purchases from customers in 6 months and cltv as exp_sales_6_month 
#and add it to dataframe

In [41]:
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(cltv_df['frequency'],
        cltv_df['recency_cltv_weekly'],
        cltv_df['T_weekly'])

<lifetimes.BetaGeoFitter: fitted with 19945 subjects, a: 0.00, alpha: 76.40, b: 0.00, r: 3.67>

In [42]:
cltv_df["exp_sales_3_month"] = bgf.predict(4*3,
                                       cltv_df['frequency'],
                                       cltv_df['recency_cltv_weekly'],
                                       cltv_df['T_weekly'])

In [43]:
cltv_df["exp_sales_6_month"] = bgf.predict(4*6,
                                       cltv_df['frequency'],
                                       cltv_df['recency_cltv_weekly'],
                                       cltv_df['T_weekly'])

In [44]:
#Step 2
#Fit the Gamma-Gamma model. 
#cltv as exp_average_value by estimating the average value that customers will leave add to dataframe

In [45]:
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(cltv_df['frequency'], cltv_df['monetary_cltv_avg'])
cltv_df["exp_average_value"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                                                cltv_df['monetary_cltv_avg'])
cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.000000,30.714286,5.0,187.874000,0.971097,1.942193,193.632679
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.857143,225.000000,21.0,95.883333,0.982145,1.964289,96.665048
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.285714,79.000000,5.0,117.064000,0.669349,1.338699,120.967619
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.571429,21.000000,2.0,60.985000,0.698327,1.396654,67.320145
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.142857,95.571429,2.0,104.990000,0.395505,0.791010,114.325108


In [47]:
#Step 3
# Calculate 6 months CLTV and add it to the dataframe with the name cltv.
#  -Observe the 20 people with the highest Cltv value

In [48]:
cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency_cltv_weekly'],
                                   cltv_df['T_weekly'],
                                   cltv_df['monetary_cltv_avg'],
                                   time=6,
                                   freq="W",
                                   discount_rate=0.01)
cltv_df["cltv"] = cltv

In [49]:
cltv_df.sort_values("cltv",ascending=False)[:20]


,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value,cltv
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,2.857143,8.000000,4.0,1401.800000,1.090271,2.180543,1449.060468,3315.269075
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,6.142857,13.285714,11.0,758.085455,1.962666,3.925331,767.360602,3160.410472
17323,f59053e2-a503-11e9-a2fc-000d3a38a36f,51.714286,101.142857,7.0,1106.467143,0.721049,1.442098,1127.611525,1706.169181
12438,625f40a2-5bd2-11ea-98b0-000d3a38a36f,74.285714,74.714286,16.0,501.873750,1.561872,3.123744,506.166665,1658.962720
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,62.714286,67.428571,52.0,166.224615,4.644667,9.289335,166.712253,1624.874497
8868,9ce6e520-89b0-11ea-a6e7-000d3a38a36f,3.428571,34.571429,8.0,601.226250,1.261764,2.523527,611.492616,1619.074843
6402,851de3b4-8f0c-11eb-8cb8-000d3a38a36f,8.285714,9.571429,2.0,862.690000,0.791164,1.582327,923.679965,1533.506236
6666,53fe00d4-7b7a-11eb-960b-000d3a38a36f,9.714286,13.142857,17.0,259.865294,2.769924,5.539848,262.072907,1523.307866
19538,55d54d9e-8ac7-11ea-8ec0-000d3a38a36f,52.571429,58.857143,31.0,228.530000,3.075833,6.151666,229.606946,1481.990623
14858,031b2954-6d28-11eb-99c4-000d3a38a36f,14.857143,15.714286,3.0,743.586667,0.868680,1.737360,778.050370,1418.290653


In [50]:
### QUEST 4 ###
### Creating Segments by CLTV Value ###

In [51]:
#Step 1 
#Divide all your customers into 4 groups (segments) according to 6-month CLTV 
#and add the group names to the dataset.

In [52]:
cltv_df["cltv_segment"] = pd.qcut(cltv_df["cltv"], 4, labels=["D", "C", "B", "A"])
cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value,cltv,cltv_segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.000000,30.714286,5.0,187.874000,0.971097,1.942193,193.632679,394.583409,A
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.857143,225.000000,21.0,95.883333,0.982145,1.964289,96.665048,199.224428,B
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.285714,79.000000,5.0,117.064000,0.669349,1.338699,120.967619,169.910284,B
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.571429,21.000000,2.0,60.985000,0.698327,1.396654,67.320145,98.650984,D
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.142857,95.571429,2.0,104.990000,0.395505,0.791010,114.325108,94.883659,D
